In [100]:
import pandas as pd

In [101]:
data=pd.read_csv('accepted_2007_to_2018Q4.csv')

/Users/lebawilamowsky/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [102]:
data=data[(data['loan_status']=='Fully Paid')| (data['loan_status']=='Charged Off')]

In [103]:
data.shape

(1345310, 151)

In [104]:
data=data[data['funded_amnt_inv']==data['loan_amnt']]

In [105]:
data.shape

(1230694, 151)

In [106]:
#columns that are known at loan origination and MAY be useful for model
data=data[['acc_open_past_24mths','addr_state','all_util','annual_inc','application_type','avg_cur_bal',
           'delinq_2yrs','dti','emp_length',
      'fico_range_high','fico_range_low','funded_amnt_inv','grade','home_ownership', 
      'inq_fi','initial_list_status','inq_last_12m','inq_last_6mths','installment','int_rate',
      'issue_d','loan_amnt','loan_status','mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op',
           'mths_since_last_delinq',
      'mths_since_last_major_derog','mths_since_last_record','mths_since_rcnt_il','mths_since_recent_bc',
      'mths_since_recent_bc_dlq','mths_since_recent_inq','mths_since_recent_revol_delinq','mort_acc',
      'num_accts_ever_120_pd','num_actv_bc_tl','num_actv_rev_tl','num_bc_sats','num_bc_tl',
      'num_il_tl','num_op_rev_tl','num_rev_accts','num_rev_tl_bal_gt_0','num_sats','num_tl_90g_dpd_24m',
      'num_tl_op_past_12m','open_acc','open_acc_6m','open_il_12m','open_il_24m','open_act_il',
      'open_rv_12m','open_rv_24m','pct_tl_nvr_dlq','percent_bc_gt_75','revol_bal','revol_util',
      'pub_rec','pub_rec_bankruptcies','purpose','sub_grade','tax_liens','term','tot_coll_amt',
      'total_acc','total_pymnt_inv','total_cu_tl','verification_status']]
      

In [107]:
data.shape

(1230694, 68)

In [108]:
data = data.sort_values(by='issue_d')

In [109]:
data[['issue_d']].sample(5)

,issue_d
396772,Jan-2015
212480,Jul-2015
297137,May-2015
82499,Oct-2015
1685620,Mar-2017


In [110]:
data['issue_d'].dtypes

dtype('O')

In [111]:
data['issue_d'].head(10)

1650791    Apr-2008
1650758    Apr-2008
1650749    Apr-2008
1650743    Apr-2008
1650739    Apr-2008
1650723    Apr-2008
1648733    Apr-2009
1648869    Apr-2009
1648852    Apr-2009
1648837    Apr-2009
Name: issue_d, dtype: object

In [112]:
data['year']=data['issue_d'].astype(str).str[-4:]

In [113]:
data[['issue_d','year']].sample(10)

,issue_d,year
387765,Jan-2015,2015
328286,Apr-2015,2015
1077637,Feb-2016,2016
2229429,Oct-2016,2016
1789900,Oct-2013,2013
310331,Apr-2015,2015
558807,Sep-2017,2017
458953,Mar-2018,2018
358372,Mar-2015,2015
1126589,Nov-2014,2014


In [114]:
data['issue_d'].value_counts()

Mar-2016    44717
Oct-2015    39851
Dec-2015    36547
Jul-2015    36230
Oct-2014    34805
Feb-2016    33436
Jan-2015    31112
Apr-2015    30717
Nov-2015    30005
Aug-2015    28131
May-2015    27594
Jan-2016    27556
Jul-2014    26637
Jun-2015    24193
Apr-2016    22987
Nov-2014    22350
Sep-2015    22094
Aug-2016    21953
Mar-2015    21289
Jul-2016    20917
Feb-2015    20794
Jun-2016    20027
Oct-2016    18336
Dec-2016    17967
May-2016    17310
Nov-2016    17285
Aug-2014    16848
Apr-2014    16467
Mar-2017    16431
Sep-2016    16192
Jan-2017    15957
May-2017    15745
May-2014    15285
Jun-2014    15249
Aug-2017    15140
Jun-2017    14861
Jul-2017    14625
Mar-2014    14571
Dec-2013    13186
Sep-2017    13105
Feb-2017    13071
Feb-2014    13009
Jan-2014    13004
Nov-2013    12978
Apr-2017    12739
Oct-2013    12372
Nov-2017    11856
Oct-2017    11629
Sep-2013    11275
Aug-2013    10576
Jul-2013    10471
Jun-2013     9598
Sep-2014     9538
May-2013     9216
Dec-2017     9017
Dec-2014  

In [115]:
#a lot of data missing feature before 2016 so get rid of those rows
#so we are analyzing data 2016-2018
data=data[(data['year']=='2016')| (data['year']=='2017')| (data['year']=='2018')]

In [116]:
data['application_type'].value_counts()

Individual    473119
Joint App      24080
Name: application_type, dtype: int64

In [117]:
#get rid of joint applications because we are analyzing
#attributes of an individual borrower
data=data[data['application_type']=='Individual']

In [118]:
data.shape

(473119, 69)

In [119]:
pd.set_option('display.max_rows', None)
pd.isnull(data).sum(axis=0)

acc_open_past_24mths                   0
addr_state                             0
all_util                              87
annual_inc                             0
application_type                       0
avg_cur_bal                            7
delinq_2yrs                            0
dti                                    0
emp_length                         30731
fico_range_high                        0
fico_range_low                         0
funded_amnt_inv                        0
grade                                  0
home_ownership                         0
inq_fi                                49
initial_list_status                    0
inq_last_12m                          50
inq_last_6mths                         1
installment                            0
int_rate                               0
issue_d                                0
loan_amnt                              0
loan_status                            0
mo_sin_old_il_acct                 12468
mo_sin_old_rev_t

In [120]:
#percent of data missing
(pd.isnull(data).sum(axis=0)/data.shape[0])*100

acc_open_past_24mths               0.000000
addr_state                         0.000000
all_util                           0.018389
annual_inc                         0.000000
application_type                   0.000000
avg_cur_bal                        0.001480
delinq_2yrs                        0.000000
dti                                0.000000
emp_length                         6.495406
fico_range_high                    0.000000
fico_range_low                     0.000000
funded_amnt_inv                    0.000000
grade                              0.000000
home_ownership                     0.000000
inq_fi                             0.010357
initial_list_status                0.000000
inq_last_12m                       0.010568
inq_last_6mths                     0.000211
installment                        0.000000
int_rate                           0.000000
issue_d                            0.000000
loan_amnt                          0.000000
loan_status                     

In [121]:
#we can drop many of the features that have so much missing data because anyway the
#FICO score and grade takes into account these features. They're not really necessary.
data.drop(['mo_sin_old_il_acct','mths_since_last_delinq','mths_since_last_delinq',
          'mths_since_last_major_derog',
          'mths_since_last_record','mths_since_rcnt_il','mths_since_recent_bc',
          'mths_since_recent_bc_dlq','percent_bc_gt_75','mths_since_recent_inq',
          'mths_since_recent_revol_delinq','percent_bc_gt_75'],axis=1, inplace=True)

In [122]:
data.shape

(473119, 59)

In [123]:
data.isnull().sum()

acc_open_past_24mths         0
addr_state                   0
all_util                    87
annual_inc                   0
application_type             0
avg_cur_bal                  7
delinq_2yrs                  0
dti                          0
emp_length               30731
fico_range_high              0
fico_range_low               0
funded_amnt_inv              0
grade                        0
home_ownership               0
inq_fi                      49
initial_list_status          0
inq_last_12m                50
inq_last_6mths               1
installment                  0
int_rate                     0
issue_d                      0
loan_amnt                    0
loan_status                  0
mo_sin_old_rev_tl_op         0
mort_acc                     0
num_accts_ever_120_pd        0
num_actv_bc_tl               0
num_actv_rev_tl              0
num_bc_sats                  0
num_bc_tl                    0
num_il_tl                    0
num_op_rev_tl                0
num_rev_

In [124]:
#get rid of rows where these are na, since na randomly. and only a tiny portion 
#of the rows anyway, so not a big deal
data.dropna(subset=['all_util','avg_cur_bal','dti','inq_fi','inq_last_12m','inq_last_6mths',
            'open_acc_6m','open_il_12m','open_il_24m','open_act_il','open_rv_12m','open_rv_24m',
            'pct_tl_nvr_dlq','revol_util','total_cu_tl'],inplace=True)

In [125]:
data.shape

(472714, 59)

In [126]:
data[data['emp_length'].isnull()==True].sample(5)

,acc_open_past_24mths,addr_state,all_util,annual_inc,application_type,avg_cur_bal,delinq_2yrs,dti,emp_length,fico_range_high,...,purpose,sub_grade,tax_liens,term,tot_coll_amt,total_acc,total_pymnt_inv,total_cu_tl,verification_status,year
2177931,6.0,TX,29.0,30600.0,Individual,13800.0,0.0,6.71,NaN,704.0,...,debt_consolidation,B3,0.0,36 months,0.0,21.0,4259.53,1.0,Verified,2016
1048302,4.0,AZ,58.0,94683.0,Individual,3837.0,0.0,13.30,NaN,664.0,...,home_improvement,D1,0.0,60 months,0.0,26.0,15162.49,2.0,Verified,2016
962540,2.0,TN,60.0,52500.0,Individual,8113.0,0.0,14.65,NaN,684.0,...,debt_consolidation,D1,0.0,36 months,540.0,26.0,30309.33,2.0,Not Verified,2017
713573,1.0,CA,60.0,16638.0,Individual,583.0,0.0,10.17,NaN,669.0,...,debt_consolidation,B3,0.0,36 months,0.0,15.0,967.71,0.0,Verified,2016
2161713,7.0,MI,32.0,24540.0,Individual,3290.0,0.0,31.26,NaN,674.0,...,debt_consolidation,D2,0.0,36 months,0.0,31.0,12052.30,3.0,Verified,2016


In [127]:
data['emp_length'].fillna('None', inplace=True)

In [128]:
pd.isnull(data).sum(axis=0)

acc_open_past_24mths     0
addr_state               0
all_util                 0
annual_inc               0
application_type         0
avg_cur_bal              0
delinq_2yrs              0
dti                      0
emp_length               0
fico_range_high          0
fico_range_low           0
funded_amnt_inv          0
grade                    0
home_ownership           0
inq_fi                   0
initial_list_status      0
inq_last_12m             0
inq_last_6mths           0
installment              0
int_rate                 0
issue_d                  0
loan_amnt                0
loan_status              0
mo_sin_old_rev_tl_op     0
mort_acc                 0
num_accts_ever_120_pd    0
num_actv_bc_tl           0
num_actv_rev_tl          0
num_bc_sats              0
num_bc_tl                0
num_il_tl                0
num_op_rev_tl            0
num_rev_accts            0
num_rev_tl_bal_gt_0      0
num_sats                 0
num_tl_90g_dpd_24m       0
num_tl_op_past_12m       0
o

In [129]:
data.dtypes

acc_open_past_24mths     float64
addr_state                object
all_util                 float64
annual_inc               float64
application_type          object
avg_cur_bal              float64
delinq_2yrs              float64
dti                      float64
emp_length                object
fico_range_high          float64
fico_range_low           float64
funded_amnt_inv          float64
grade                     object
home_ownership            object
inq_fi                   float64
initial_list_status       object
inq_last_12m             float64
inq_last_6mths           float64
installment              float64
int_rate                 float64
issue_d                   object
loan_amnt                float64
loan_status               object
mo_sin_old_rev_tl_op     float64
mort_acc                 float64
num_accts_ever_120_pd    float64
num_actv_bc_tl           float64
num_actv_rev_tl          float64
num_bc_sats              float64
num_bc_tl                float64
num_il_tl 

In [130]:
data['avg_fico_score']=(data['fico_range_high']+data['fico_range_low'])/2

In [131]:
data[['avg_fico_score','fico_range_high','fico_range_low']].sample(10)

,avg_fico_score,fico_range_high,fico_range_low
1656117,707.0,709.0,705.0
699959,692.0,694.0,690.0
1959028,677.0,679.0,675.0
1745732,692.0,694.0,690.0
714972,722.0,724.0,720.0
1939510,717.0,719.0,715.0
2118152,687.0,689.0,685.0
2242163,667.0,669.0,665.0
2242038,842.0,844.0,840.0
599085,757.0,759.0,755.0


In [132]:
data['Profit_Percentage_Inv']= round(((data['total_pymnt_inv']-data['funded_amnt_inv']) / data['funded_amnt_inv'])*100)

In [133]:
data[['funded_amnt_inv','total_pymnt_inv','loan_status','Profit_Percentage_Inv']].sample(50)

,funded_amnt_inv,total_pymnt_inv,loan_status,Profit_Percentage_Inv
1083311,20000.0,22940.00,Fully Paid,15.0
1109091,24000.0,14990.10,Charged Off,-38.0
908655,5600.0,6047.53,Fully Paid,8.0
695244,15000.0,16624.75,Fully Paid,11.0
1980920,2100.0,1442.48,Charged Off,-31.0
668805,20000.0,13443.89,Charged Off,-33.0
1963395,7000.0,7212.83,Fully Paid,3.0
2133444,5000.0,2278.89,Charged Off,-54.0
1709515,9050.0,10373.55,Fully Paid,15.0
1040356,7500.0,7823.32,Fully Paid,4.0


In [134]:
data['Profit_Percentage_Inv'].value_counts().sort_index()

-100.0      619
-99.0        15
-98.0       149
-97.0       510
-96.0       299
-95.0       244
-94.0       397
-93.0       786
-92.0       406
-91.0       462
-90.0       965
-89.0       678
-88.0       568
-87.0      1049
-86.0      1250
-85.0       850
-84.0      1261
-83.0      1419
-82.0      1180
-81.0      1353
-80.0      1698
-79.0      1388
-78.0      1428
-77.0      1727
-76.0      1589
-75.0      1576
-74.0      1817
-73.0      1696
-72.0      1615
-71.0      1746
-70.0      1717
-69.0      1767
-68.0      1844
-67.0      1796
-66.0      1794
-65.0      1747
-64.0      1824
-63.0      1802
-62.0      1717
-61.0      1703
-60.0      1811
-59.0      1738
-58.0      1581
-57.0      1703
-56.0      1644
-55.0      1454
-54.0      1767
-53.0      1572
-52.0      1590
-51.0      1522
-50.0      1474
-49.0      1509
-48.0      1379
-47.0      1369
-46.0      1364
-45.0      1353
-44.0      1301
-43.0      1232
-42.0      1271
-41.0      1191
-40.0      1126
-39.0      1119
-38.0   

In [195]:
data.shape

(472714, 71)

In [198]:
def f(df):
    if df['Profit_Percentage_Inv'] < -66:
        val = '-100% to -67%'
    elif df['Profit_Percentage_Inv'] < -33:
        val = '-66 to -34%'
    elif df['Profit_Percentage_Inv'] < 0:
        val = '-33% to -1%'
    elif df['Profit_Percentage_Inv'] < 7:
        val= '0% to 6%'
    elif df['Profit_Percentage_Inv'] < 14:
        val= '7% to 13%'
    elif df['Profit_Percentage_Inv'] < 18:
        val = '14% to 17%'
    elif df['Profit_Percentage_Inv'] <23:
        val = '18% to 22%'
    elif df['Profit_Percentage_Inv'] <33:
        val = '23% to 32%'
    else:
        val = 'greater than 32%'
    return val

In [199]:
data['Profit_Percentage_Inv_Range']= data.apply(f, axis=1)

In [200]:
data[['Profit_Percentage_Inv_Range','Profit_Percentage_Inv']].sample(20)

,Profit_Percentage_Inv_Range,Profit_Percentage_Inv
1943124,7% to 13%,8.0
1730427,-66 to -34%,-56.0
1570611,0% to 6%,3.0
942597,0% to 6%,1.0
2103244,7% to 13%,7.0
1501852,-100% to -67%,-100.0
1097372,-66 to -34%,-43.0
1082625,7% to 13%,8.0
1991310,14% to 17%,17.0
2166084,7% to 13%,11.0


In [201]:
data['Profit_Percentage_Inv_Range'].value_counts()

7% to 13%           139108
0% to 6%            107381
14% to 17%           54617
-66 to -34%          47855
18% to 22%           38005
-100% to -67%        37864
23% to 32%           23915
-33% to -1%          17142
greater than 32%      6827
Name: Profit_Percentage_Inv_Range, dtype: int64

In [202]:
#need to encode 'addr_state','emp_length','grade', 'home_ownership','purpose', 
#               'sub_grade', 'term', 'verification_status'

In [203]:
data['emp_length'].value_counts()

10+ years    160559
2 years       44187
3 years       38893
< 1 year      34805
1 year        32323
None          30693
5 years       29454
4 years       28906
6 years       20693
8 years       18278
9 years       17514
7 years       16409
Name: emp_length, dtype: int64

In [142]:
#encode emp_length
import numpy as np
conditions = [
    (data['emp_length'] == '1 year'),
    (data['emp_length'] == '10+ years'),
    (data['emp_length'] == '2 years'),
    (data['emp_length'] == '3 years'),
    (data['emp_length'] == '4 years'),
    (data['emp_length'] == '5 years'),
    (data['emp_length'] == '6 years'),
    (data['emp_length'] == '7 years'),
    (data['emp_length'] == '8 years'),
    (data['emp_length'] == '9 years'),
    (data['emp_length'] == '< 1 year'),
    (data['emp_length']=='None')
    ]
values = [1, 10, 2, 3, 4, 5, 6, 7, 8, 9, .5, 0]
data['emp_length_enc']=np.select(conditions, values)

In [143]:
#encode grade
data['grade'].value_counts()

C    143606
B    139012
A     82331
D     67075
E     27710
F      9569
G      3411
Name: grade, dtype: int64

In [144]:
conditions = [
    (data['grade'] == 'A'),
    (data['grade'] == 'B'),
    (data['grade'] == 'C'),
    (data['grade'] == 'D'),
    (data['grade'] == 'E'),
    (data['grade'] == 'F'),
    (data['grade'] == 'G')
    ]

values = [1, 2, 3, 4, 5, 6, 7]
data['grade_enc']=np.select(conditions, values)

In [147]:
data['sub_grade'].value_counts().sort_index()

A1    19628
A2    13623
A3    13474
A4    16524
A5    19082
B1    25790
B2    24943
B3    26357
B4    28745
B5    33177
C1    33120
C2    28456
C3    26902
C4    27940
C5    27188
D1    16803
D2    15825
D3    13595
D4    11348
D5     9504
E1     6809
E2     5755
E3     5214
E4     4743
E5     5189
F1     2997
F2     2029
F3     1738
F4     1476
F5     1329
G1     1008
G2      702
G3      610
G4      585
G5      506
Name: sub_grade, dtype: int64

In [148]:
conditions = [
    (data['sub_grade'] == 'A1'),
    (data['sub_grade'] == 'A2'),
    (data['sub_grade'] == 'A3'),
    (data['sub_grade'] == 'A4'),
    (data['sub_grade'] == 'A5'),
    (data['sub_grade'] == 'B1'),
    (data['sub_grade'] == 'B2'),
    (data['sub_grade'] == 'B3'),
    (data['sub_grade'] == 'B4'),
    (data['sub_grade'] == 'B5'),
    (data['sub_grade'] == 'C1'),
    (data['sub_grade'] == 'C2'),
    (data['sub_grade'] == 'C3'),
    (data['sub_grade'] == 'C4'),
    (data['sub_grade'] == 'C5'),
    (data['sub_grade'] == 'D1'),
    (data['sub_grade'] == 'D2'),
    (data['sub_grade'] == 'D3'),
    (data['sub_grade'] == 'D4'),
    (data['sub_grade'] == 'D5'),
    (data['sub_grade'] == 'E1'),
    (data['sub_grade'] == 'E2'),
    (data['sub_grade'] == 'E3'),
    (data['sub_grade'] == 'E4'),
    (data['sub_grade'] == 'E5'),
    (data['sub_grade'] == 'F1'),
    (data['sub_grade'] == 'F2'),
    (data['sub_grade'] == 'F3'),
    (data['sub_grade'] == 'F4'),
    (data['sub_grade'] == 'F5'),
    (data['sub_grade'] == 'G1'),
    (data['sub_grade'] == 'G2'),
    (data['sub_grade'] == 'G3'),
    (data['sub_grade'] == 'G4'),
    (data['sub_grade'] == 'G5')
    ]
values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]
data['sub_grade_enc']=np.select(conditions, values)


In [150]:
from sklearn.preprocessing import OrdinalEncoder
ordinalencoder = OrdinalEncoder()

In [222]:
#create new columns, fill with random info because about to change it anyway
data[['addr_state_enc']]=data[['initial_list_status']]
data[['home_ownership_enc']]=data[['initial_list_status']]
data[['purpose_enc']]=data[['initial_list_status']]
data[['term_enc']]=data[['initial_list_status']]
data[['verification_status_enc']]=data[['initial_list_status']]
data[['initial_list_status_enc']]=data[['initial_list_status']]

In [223]:
data[['addr_state_enc']]=ordinalencoder.fit_transform(data[['addr_state']])
data[['home_ownership_enc']]=ordinalencoder.fit_transform(data[['home_ownership']])
data[['purpose_enc']]=ordinalencoder.fit_transform(data[['purpose']])
data[['term_enc']]=ordinalencoder.fit_transform(data[['term']])
data[['verification_status_enc']]=ordinalencoder.fit_transform(data[['verification_status']])
data[['initial_list_status_enc']]=ordinalencoder.fit_transform(data[['initial_list_status']])

In [225]:
data.columns.values

array(['acc_open_past_24mths', 'addr_state', 'all_util', 'annual_inc',
       'application_type', 'avg_cur_bal', 'delinq_2yrs', 'dti',
       'emp_length', 'fico_range_high', 'fico_range_low',
       'funded_amnt_inv', 'grade', 'home_ownership', 'inq_fi',
       'initial_list_status', 'inq_last_12m', 'inq_last_6mths',
       'installment', 'int_rate', 'issue_d', 'loan_amnt', 'loan_status',
       'mo_sin_old_rev_tl_op', 'mort_acc', 'num_accts_ever_120_pd',
       'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl',
       'num_il_tl', 'num_op_rev_tl', 'num_rev_accts',
       'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_90g_dpd_24m',
       'num_tl_op_past_12m', 'open_acc', 'open_acc_6m', 'open_il_12m',
       'open_il_24m', 'open_act_il', 'open_rv_12m', 'open_rv_24m',
       'pct_tl_nvr_dlq', 'revol_bal', 'revol_util', 'pub_rec',
       'pub_rec_bankruptcies', 'purpose', 'sub_grade', 'tax_liens',
       'term', 'tot_coll_amt', 'total_acc', 'total_pymnt_inv',
       'total_cu_

In [204]:
target=data[['Profit_Percentage_Inv_Range']]

In [226]:
features=data[['acc_open_past_24mths','all_util', 'annual_inc',
               'avg_cur_bal', 'delinq_2yrs', 'dti','inq_fi',
       'inq_last_12m', 'inq_last_6mths',
       'installment', 'int_rate', 'loan_amnt',
       'mo_sin_old_rev_tl_op', 'mort_acc', 'num_accts_ever_120_pd',
       'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl',
       'num_il_tl', 'num_op_rev_tl', 'num_rev_accts',
       'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_90g_dpd_24m',
       'num_tl_op_past_12m', 'open_acc', 'open_acc_6m', 'open_il_12m',
       'open_il_24m', 'open_act_il', 'open_rv_12m', 'open_rv_24m',
       'pct_tl_nvr_dlq', 'revol_bal', 'revol_util', 'pub_rec',
       'pub_rec_bankruptcies', 'tax_liens',
       'tot_coll_amt', 'total_acc',
       'total_cu_tl','avg_fico_score',
       'emp_length_enc', 'grade_enc', 'sub_grade_enc', 'addr_state_enc',
       'home_ownership_enc', 'purpose_enc', 'term_enc',
       'verification_status_enc','initial_list_status_enc']]

In [227]:
features.shape

(472714, 52)

In [228]:
features.dtypes

acc_open_past_24mths       float64
all_util                   float64
annual_inc                 float64
avg_cur_bal                float64
delinq_2yrs                float64
dti                        float64
inq_fi                     float64
inq_last_12m               float64
inq_last_6mths             float64
installment                float64
int_rate                   float64
loan_amnt                  float64
mo_sin_old_rev_tl_op       float64
mort_acc                   float64
num_accts_ever_120_pd      float64
num_actv_bc_tl             float64
num_actv_rev_tl            float64
num_bc_sats                float64
num_bc_tl                  float64
num_il_tl                  float64
num_op_rev_tl              float64
num_rev_accts              float64
num_rev_tl_bal_gt_0        float64
num_sats                   float64
num_tl_90g_dpd_24m         float64
num_tl_op_past_12m         float64
open_acc                   float64
open_acc_6m                float64
open_il_12m         